# Imports

In [1]:
import torch
import cgd_utils

# Cournot Simulation

## Defining Payoff Calculation

In [73]:
def player_payoffs(quantity_tensor,
                   market_demand=lambda q: 100 - q,
                   marginal_cost=lambda q: q * 10):
    """Assuming player list is ordered in order of increasing price."""
    price = torch.max(market_demand(torch.sum(quantity_tensor)),
                      torch.tensor(0., requires_grad=True))

    payoffs = []
    for i, quantity in enumerate(quantity_tensor):
        payoffs.append(- (quantity * price - marginal_cost(quantity)))
        
    return torch.stack(payoffs)

## Defining Players and Objectives

In [81]:
num_iterations = 50

# Define individual sellers and prices
p1 = torch.tensor([0.], requires_grad=True)
p2 = torch.tensor([0.], requires_grad=True)
p3 = torch.tensor([0.], requires_grad=True)

players = torch.stack([p1, p2, p3])

learning_rates = [0.1, 0.1, 0.1]

for i in range(num_iterations):
    print(players)
    payoffs = player_payoffs(players)
    updates, n_iter = cgd_utils.metamatrix_conjugate_gradient(payoffs, [p1, p2, p3], lr_list=learning_rates)
    
    for player, update in zip(players, updates):
        player.data.add_(update)
    


tensor([[0.],
        [0.],
        [0.]], grad_fn=<StackBackward>)
tensor([[7.5000],
        [7.5000],
        [7.5000]], grad_fn=<StackBackward>)
tensor([[12.5000],
        [12.5000],
        [12.5000]], grad_fn=<StackBackward>)
tensor([[15.8333],
        [15.8333],
        [15.8333]], grad_fn=<StackBackward>)
tensor([[18.0556],
        [18.0556],
        [18.0556]], grad_fn=<StackBackward>)
tensor([[19.5370],
        [19.5370],
        [19.5370]], grad_fn=<StackBackward>)
tensor([[20.5247],
        [20.5247],
        [20.5247]], grad_fn=<StackBackward>)
tensor([[21.1831],
        [21.1831],
        [21.1831]], grad_fn=<StackBackward>)
tensor([[21.6221],
        [21.6221],
        [21.6221]], grad_fn=<StackBackward>)
tensor([[21.9147],
        [21.9147],
        [21.9147]], grad_fn=<StackBackward>)
tensor([[22.1098],
        [22.1098],
        [22.1098]], grad_fn=<StackBackward>)
tensor([[22.2399],
        [22.2399],
        [22.2399]], grad_fn=<StackBackward>)
tensor([[22.3266],
   